## Import Necessary Libraries for Study

In [1]:
import os
import re
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.svm import OneClassSVM
from sklearn.decomposition import PCA

## Configuration and Application of Images for PCA + SVM

In [2]:
# Define paths
IMG_DIR = "/Users/annethsivakumar/Downloads/Left_images2/"
OUTPUT_DIR = "/Users/annethsivakumar/Downloads/classified_output/"

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Get all .png images in the directory (sorted numerically)
image_files = sorted(
    [f for f in os.listdir(IMG_DIR) if f.endswith(".png")],
    key=lambda x: int(re.findall(r'\d+', x)[0]) if re.findall(r'\d+', x) else 0
)

# PCA and SVM parameters
# Sliding-window patch size and step.
PATCH_SIZE = (32, 32)    # (width, height)
STEP_SIZE = (16, 16)    # (step_x, step_y)
# One-Class SVM parameters.
OCSVM_NU = 0.1        # Proportion of outliers you expect
OCSVM_KERNEL = 'rbf'
OCSVM_GAMMA = 'auto'

# PCA parameters.
N_COMPONENTS = 50    # Number of principal components to keep (adjust as needed)

# Color histogram parameters.
COLOR_BINS = (8, 8, 8)    # number of bins for each channel

# HOG parameters.
HOG_PARAMS = {
    'pixels_per_cell': (8, 8),
    'cells_per_block': (2, 2),
    'orientations': 9,
    'block_norm': 'L2-Hys'
}

# Filtering thresholds.
# 1) Minimum number of patches in a connected component (in patch-space).
MIN_COMPONENT_AREA_PATCHES = 10
# 2) Minimum bounding box area in pixel-space.
MIN_BBOX_AREA_PX = 2000

# Functions for extracting features
def color_histogram_features(patch_bgr, bins=(8, 8, 8)):
    """
    Compute a 3D color histogram (B, G, R) and normalize it.
    """
    hist_b = cv2.calcHist([patch_bgr], [0], None, [bins[0]], [0, 256]).flatten()
    hist_g = cv2.calcHist([patch_bgr], [1], None, [bins[1]], [0, 256]).flatten()
    hist_r = cv2.calcHist([patch_bgr], [2], None, [bins[2]], [0, 256]).flatten()
    hist_b /= (hist_b.sum() + 1e-6)
    hist_g /= (hist_g.sum() + 1e-6)
    hist_r /= (hist_r.sum() + 1e-6)
    return np.concatenate([hist_b, hist_g, hist_r])

def hog_features(patch_bgr):
    """
    Compute HOG on the grayscale version of the patch.
    """
    patch_gray = cv2.cvtColor(patch_bgr, cv2.COLOR_BGR2GRAY)
    return hog(patch_gray, **HOG_PARAMS)

def extract_patch_features(patch_bgr):
    """
    Combine color histogram + HOG into a single feature vector.
    """
    return np.concatenate([color_histogram_features(patch_bgr, COLOR_BINS), hog_features(patch_bgr)])

def sliding_window(image, step_size, window_size):
    """
    Generator yielding (x, y, patch) for each top-left corner (x, y).
    """
    max_y = image.shape[0] - window_size[1] + 1
    max_x = image.shape[1] - window_size[0] + 1
    for y in range(0, max_y, step_size[1]):
        for x in range(0, max_x, step_size[0]):
            yield (x, y, image[y:y+window_size[1], x:x+window_size[0]])

# Process each image in the directory
for img_file in image_files:
    IMG_PATH = os.path.join(IMG_DIR, img_file)
    OUTPUT_PATH = os.path.join(OUTPUT_DIR, img_file)
    
    print(f"Processing: {IMG_PATH}")

    # Load image
    img_bgr = cv2.imread(IMG_PATH)
    if img_bgr is None:
        print(f"Error loading {IMG_PATH}")
        continue

    # Extract patches and features
    features_list, coords_list = [], []
    for (x, y, patch_bgr) in sliding_window(img_bgr, STEP_SIZE, PATCH_SIZE):
        feat = extract_patch_features(patch_bgr)
        features_list.append(feat)
        coords_list.append((x, y))

    features_array = np.array(features_list, dtype=np.float32)
    if features_array.shape[0] == 0:
        print(f"No valid patches extracted from {IMG_PATH}")
        continue

    # Apply PCA
    n_features = features_array.shape[1]
    n_components = min(N_COMPONENTS, n_features)
    pca = PCA(n_components=n_components, random_state=42)
    features_array_pca = pca.fit_transform(features_array)

    # Train One-Class SVM and predict anomalies
    ocsvm = OneClassSVM(nu=OCSVM_NU, kernel=OCSVM_KERNEL, gamma=OCSVM_GAMMA)
    ocsvm.fit(features_array_pca)
    predictions = ocsvm.predict(features_array_pca)  # +1 = normal, -1 = outlier

    # Convert predictions to a 2D mask
    img_h, img_w = img_bgr.shape[:2]
    grid_rows = (img_h - PATCH_SIZE[1]) // STEP_SIZE[1] + 1
    grid_cols = (img_w - PATCH_SIZE[0]) // STEP_SIZE[0] + 1
    anomaly_mask = np.zeros((grid_rows, grid_cols), dtype=np.uint8)

    for i, pred in enumerate(predictions):
        if pred == -1:
            row = i // grid_cols
            col = i % grid_cols
            anomaly_mask[row, col] = 255

    # Apply morphological opening
    kernel = np.ones((3, 3), np.uint8)
    anomaly_mask = cv2.morphologyEx(anomaly_mask, cv2.MORPH_OPEN, kernel)

    # Connected components analysis
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(anomaly_mask, connectivity=8)
    annotated_img = img_bgr.copy()
    object_count = 0

    for label_id in range(1, num_labels):
        component_area = stats[label_id, cv2.CC_STAT_AREA]
        if component_area < MIN_COMPONENT_AREA_PATCHES:
            continue

        x_patch = stats[label_id, cv2.CC_STAT_LEFT]
        y_patch = stats[label_id, cv2.CC_STAT_TOP]
        w_patch = stats[label_id, cv2.CC_STAT_WIDTH]
        h_patch = stats[label_id, cv2.CC_STAT_HEIGHT]

        x_min_pixel = x_patch * STEP_SIZE[0]
        y_min_pixel = y_patch * STEP_SIZE[1]
        x_max_pixel = (x_patch + w_patch - 1) * STEP_SIZE[0] + PATCH_SIZE[0]
        y_max_pixel = (y_patch + h_patch - 1) * STEP_SIZE[1] + PATCH_SIZE[1]

        bbox_width = x_max_pixel - x_min_pixel
        bbox_height = y_max_pixel - y_min_pixel
        bbox_area = bbox_width * bbox_height

        if bbox_area < MIN_BBOX_AREA_PX:
            continue

        # Draw bounding box
        object_count += 1
        cv2.rectangle(annotated_img, (x_min_pixel, y_min_pixel), (x_max_pixel, y_max_pixel), (0, 0, 255), 2)

        # Label the bounding box
        label_text = f"Object #{object_count}"
        text_x = x_min_pixel
        text_y = max(y_min_pixel - 5, 15)
        cv2.putText(annotated_img, label_text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    # Save annotated image
    success = cv2.imwrite(OUTPUT_PATH, annotated_img)
    if success:
        print(f"Saved annotated image to: {OUTPUT_PATH}")
    else:
        print("Error: Could not write image. Check OUTPUT_PATH and extension.")

print("Processing complete.")

Processing: /Users/annethsivakumar/Downloads/Left_images2/001640.png
Saved annotated image to: /Users/annethsivakumar/Downloads/classified_output/001640.png
Processing: /Users/annethsivakumar/Downloads/Left_images2/001641.png
Saved annotated image to: /Users/annethsivakumar/Downloads/classified_output/001641.png
Processing: /Users/annethsivakumar/Downloads/Left_images2/001642.png
Saved annotated image to: /Users/annethsivakumar/Downloads/classified_output/001642.png
Processing: /Users/annethsivakumar/Downloads/Left_images2/001643.png
Saved annotated image to: /Users/annethsivakumar/Downloads/classified_output/001643.png
Processing: /Users/annethsivakumar/Downloads/Left_images2/001644.png
Saved annotated image to: /Users/annethsivakumar/Downloads/classified_output/001644.png
Processing: /Users/annethsivakumar/Downloads/Left_images2/001645.png
Saved annotated image to: /Users/annethsivakumar/Downloads/classified_output/001645.png
Processing: /Users/annethsivakumar/Downloads/Left_images2/